In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
import dotenv
import langchain_huggingface

_ = dotenv.load_dotenv(dotenv_path="../.env", override=True)

llm_endpoint = langchain_huggingface.HuggingFaceEndpoint(
    huggingfacehub_api_token=os.environ["HF_TOKEN"],
    repo_id="deepseek-ai/DeepSeek-V3.2",
    task="text-generation",
    provider="auto", temperature=0.1
)
llm = langchain_huggingface.ChatHuggingFace(llm=llm_endpoint)

In [ ]:
import pydantic

class GraphState(pydantic.BaseModel):
    url: str
    raw_text: str | None = None
    holiday_info: dict | None = None

class HolidayInfo(pydantic.BaseModel):
    country_name: str = pydantic.Field(description= "The name of the country")
    holiday: str = pydantic.Field(description="The name of the holiday")
    date: str = pydantic.Field(description="The date of the holiday")

In [ ]:
from langchain_community import document_loaders, document_transformers

def scrape_page(state: GraphState) -> GraphState:
    loader = document_loaders.AsyncChromiumLoader(
        urls=[state.url]
    )
    docs = loader.load()
    transformer = document_transformers.BeautifulSoupTransformer()
    cleaned_docs = transformer.transform_documents(
        documents=docs,
        tags_to_extract=["li"]
    )

    return state.copy(update={
        "raw_text": cleaned_docs[0].page_content
    })

def extract

In [ ]:
from langchain import tools
from langchain_community import document_loaders, document_transformers
import pydantic

@tools.tool("get_holiday")
def get_holiday(url: str) -> str:
    """
    Scrapes a URL and returns a holiday.
    """
    # Initialize Loader
    loader = document_loaders.AsyncChromiumLoader(urls=[url])
    docs = loader.load()
    # transform HTML to clean text
    bs_transformer = document_transformers.BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(documents=docs,
        tags_to_extract=["li"])
    
    return docs_transformed[0].page_content

class HolidayInfo(pydantic.BaseModel):
    """
    Structured holiday information.
    """
    country_name: str = pydantic.Field(description= "The name of the country")
    holiday: str = pydantic.Field(description="The name of the holiday")
    date: str = pydantic.Field(description="The date of the holiday")

In [ ]:
from langchain_core import prompts
from langchain import agents
# from langgraph import prebuilt

agent = agents.create_agent(
    model=llm,
    tools=[get_holiday],
    response_format=HolidayInfo,
    debug=True
)

# prompt_template = prompts.ChatPromptTemplate.from_messages(messages=[
#     ("system", "You're a helpful research assistant. Use tools to scrape web data."),
#     ("human", "Go to {url} and extract the country name from the website")
# ])

# chain = prompt_template | agent

url = "https://www.timeanddate.com/holidays/?allcountries"
messages = [
    ("system", "You're a helpful research assistant. Use tools to scrape web data."),
    ("human", f"Go to {url} and extract the country name from the website")
]
response = agent.invoke({"messages": messages})
response["structured_response"]